<a href="https://colab.research.google.com/github/ascarrambad/ml-21-22/blob/main/01_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning SP 2021/2022

Prof. Cesare Alippi<br>
Giorgia Adorni ([`giorgia.adorni@usi.ch`](mailto:giorgia.adorni@usi.ch))<br>
Luca Butera ([`luca.butera@usi.ch`](mailto:luca.butera@usi.ch))<br>
Matteo Riva ([`matteo.riva@usi.ch`](mailto:matteo.riva@usi.ch))

---

# Lab 01: Linear Regression

# 01.A) Let's collect some data
... or let someone do it for us :)

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
X = boston.data
y = boston.target

print(boston.DESCR)
print("X shape:", X.shape)
print("y shape:", y.shape)

Data set of $n=506$ observations $\{(\mathbf x_1, y_1), (\mathbf x_2, y_2) ,\dots,(\mathbf x_n, y_n)\}$, where $\mathbf x_i\in\mathbb R^{d}$, with $d=13$ and $y_i\in\mathbb R$. All the observations are stack to form

$$
X = \left[ 
\begin{array}{c}
\mathbf x_1\\
\mathbf x_2\\
\vdots \\
\mathbf x_n
\end{array}
\right] 
\in \mathbb{R}^{n\times d},
\qquad 
Y = \left[ 
\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{array}
\right] 
\in \mathbb{R}^{n}
$$


In [ ]:
# Let's consider only the RM index, for now.
x = X[:, 5]  

import matplotlib.pyplot as plt
plt.scatter(x, y, c='k', marker='.');
plt.xlabel("RM");
plt.ylabel("thousand dollars");

# 01.B) System model

We assume there is a function $g(x)$ that links RM index to the house price:
$$
y = g(x) + \eta
$$
where $\eta \sim N(0, \sigma^2_\eta)$.

Every line in the plane (except the vertical ones) can be written in the form
$$f(x; \boldsymbol \theta) = \theta_0 + \theta_1 x$$ 
with $\boldsymbol \theta = (\theta_0, \theta_1)$ and $\theta_0,\theta_1 \in\mathbb R$.

We also assume that $g(.)$ is linear, that is, there exists $\boldsymbol \theta^o$ so that $ g(x) = f(x; \boldsymbol \theta^o)$.


In [ ]:
import numpy as np

def my_lin_fun(x, theta):
    y = theta[0] + x * theta[1]
    return y

# some guesses
xx = np.array([4., 9.])
plt.plot(xx, my_lin_fun(xx, [.1, 3.]))
plt.plot(xx, my_lin_fun(xx, [-1., 5.]))
plt.scatter(x, y, c='k', marker='.');

# 01.C) Model approximation

Given a new value of $x$ our best prediction for $y$ is
$$\hat y = E[y] = E[f(x; \boldsymbol \theta^o) + \eta] = f(x; \boldsymbol \theta^o) + E[\eta] = f(x; \boldsymbol \theta^o).$$

Since ${\boldsymbol \theta^o}$ is unknown, we estimate it from the data, by minimising 
$$ \hat{\boldsymbol \theta} = \mathop{\mathrm{arg\,min}}_{\boldsymbol \theta} \sum_{i=1}^n \left\lVert y_i - f(x;\boldsymbol \theta) \right\rVert^2_2 $$

Finally, we predict new house prices with 
$$\hat y = f\left(x; \hat{\boldsymbol \theta}\right).$$

## 01.D) Parameter estimation

__Data in compact form:__ Prepending a '1' to each $\mathbf x$, then any 
$f(\mathbf x,\boldsymbol \theta)= \mathbf x^\top \boldsymbol \theta$. In fact, $\mathbf x^\top \boldsymbol \theta = \theta_0 1 +\theta_1 x$.

We showed in class that the solution $\hat{\boldsymbol \theta}$ to $\mathop{\mathrm{arg\,min}}_{\boldsymbol \theta} \sum_{i=1}^n \left\lVert y_i - f(x;\boldsymbol \theta) \right\rVert^2_2$ 
can be found by solving the linear system
$$
X^\top Y = X^\top X \boldsymbol \theta
$$
with respect to the $\boldsymbol \theta$.

In our 1-D case,
$$
X = \left[ 
\begin{array}{c}
1, x_1 \\
1, x_2 \\
\vdots \\
1, x_n
\end{array}
\right] 
\in \mathbb{R}^{n\times 2},
\qquad 
Y = \left[ 
\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{array}
\right] 
\in \mathbb{R}^{n}
$$

In [ ]:
# Solving the linear system
x_col_vec = x.reshape(-1, 1)
ones_col_vec = np.ones(shape=(y.shape[0], 1))
X = np.hstack((ones_col_vec, x_col_vec))
theta_hat = np.linalg.solve(a=X.T.dot(X), b=X.T.dot(y))  # solves ax=b wrt x
print(theta_hat)

In [ ]:
# plot the result
plt.plot(xx, my_lin_fun(xx, theta_hat), 'g')
plt.scatter(x, y, c='k', marker='.');

In [ ]:
# There are also libraries for this
from sklearn.linear_model import LinearRegression

# init the model
lr = LinearRegression(fit_intercept=False)  

# estimate parameters
lr.fit(X, y)
theta_hat2 = lr.coef_

print('theta1 = {}'.format(theta_hat))
print('theta2 = {}'.format(theta_hat2))

In [ ]:
# We can also avoid creating a column of ones
lr = LinearRegression(fit_intercept=True)  # default is True  
lr.fit(x_col_vec, y)
theta_hat3 = [lr.intercept_, lr.coef_[0]]
print('theta3 = {}'.format(theta_hat3))

# 01.E) More generally 

## i. Multidimensional data

When the input (regressor) $\mathbf x=[x_1,\dots,x_d]$ is $d$-dimensional, then
$$y = f(\mathbf x, \boldsymbol \theta)  + \eta = \mathbf x^\top \boldsymbol \theta = x_1 \theta_1 + x_2 \theta_2+ \dots + x_d\theta_d + \eta.$$
with $\mathbf x,\boldsymbol \theta \in\mathbb R^d$.

When also the output (target) $\mathbf y$ is multidimensional:
$$\mathbf y = f(\mathbf x, \Theta) +\eta= \mathbf x^\top \Theta +\eta$$
with $\mathbf y\in\mathbb R^f$, $\Theta \in\mathbb R^{d\times f}$ is a matrix and $\eta\sim N(0,I\sigma_\eta)$.


In [ ]:
# We can also avoid creating a column of ones
lr_1d = LinearRegression()  
lr_1d.fit(x_col_vec, y)
y_pred_1d = lr_1d.predict(x_col_vec)
err_1d = ((y_pred_1d - y)**2).sum()

# Let's try adding LSTAT...
lr_2d = LinearRegression()  
X2 = np.vstack((boston.data[:, 5], boston.data[:, 12])).T
lr_2d.fit(X2, y)
y_pred_2d = lr_2d.predict(X2)
err_2d = ((y_pred_2d - y)**2).sum()

# ... and finally with all the features 
lr_md = LinearRegression()  
Xall = boston.data
lr_md.fit(Xall, y)
y_pred_md = lr_md.predict(Xall)
err_md = ((y_pred_md - y)**2).sum()

print(err_1d)
print(err_2d)
print(err_md)

In [ ]:
# plot the result
%matplotlib inline

# 1-d
xx = np.array([3., 10.])
plt.plot(x_col_vec, y_pred_1d, 'g')
plt.scatter(x_col_vec, y, c='k', marker='.');

# 2-d
from mpl_toolkits.mplot3d import Axes3D  # this is necessary for 3-d plots 
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d');
ax.view_init(elev=10., azim=0)
ax.set_xlabel('x_5')
ax.set_ylabel('x_12')
ax.set_zlabel('y')
ax.plot_trisurf(X2[:, 0], X2[:, 1], y_pred_2d, alpha=0.3, label='est fun');
ax.scatter(X2[:, 0], X2[:, 1], y, c='k', marker='.');

## ii. 'Linear' means linear in the parameters

The regressor can be of the form 
$$\boldsymbol \phi(\mathbf x) = [\phi_1(\mathbf x), \phi_2(\mathbf x), \dots, \phi_m(\mathbf x)]$$ 
for any collection of functions $\phi_1,\dots,\phi_m$. 
Function $f$ become
$$f(\mathbf x, \boldsymbol \theta) = \boldsymbol \theta^\top\boldsymbol \phi(\mathbf x) =  \theta_1 \phi_1(\mathbf x) + \theta_2 \phi_d(\mathbf x) + \dots + \theta_d \phi_d(\mathbf x).$$




### Example: Polynomials

$$f(x;\boldsymbol \theta) = \theta_0 + x \theta_1 + x^2 \theta_2 + \dots + x^d \theta_d$$ 


In [ ]:
def pol_fun(x):
    return -1 -x - .1 * x**2 + .5*x**3

# generate data
n = 100
X = np.linspace(-1, 2, n).reshape(n,1) 
interval = np.linspace(-1, 2, 100).reshape(100,1)
sigma = 0.3
eta = np.random.normal(loc=0, scale=sigma, size=(n,1))
Y = pol_fun(X) + eta

# create regressor
degree = 1

# generate polynomial features
from sklearn.preprocessing import PolynomialFeatures 
pol_feat = PolynomialFeatures(degree=degree, include_bias=False) 
Xpol = pol_feat.fit_transform(X)

# estimate parameter
lr = LinearRegression()
lr.fit(Xpol, Y)
Y_est = lr.predict(pol_feat.transform(interval[:,...]))

# plot results
plt.plot(interval, pol_fun(interval), label='true fun')
plt.scatter(X, Y, label='noisy data', c='k', marker='.', alpha=0.5)
plt.plot(interval, Y_est, label='est fun')
plt.ylim((-2, 0.8))
plt.legend()


# 01.F) Regularizations

## 1. Ridge regression

In [ ]:
# generate data [200 100 50 20 10]
n = 200
X = np.linspace(-1, 2, n).reshape(n,1) 
sigma = 0.3
eps = np.random.normal(loc=0, scale=sigma, size=(n,1))
Y = pol_fun(X) + eps

# create regressor
degree = 5
pol_feat = PolynomialFeatures(degree=degree, include_bias=False) 
Xpol = pol_feat.fit_transform(X)

# linear regression
lr = LinearRegression()
lr.fit(Xpol, Y)
Y_est = lr.predict(Xpol)

# ridge regression
from sklearn.linear_model import Ridge 
rid = Ridge()
rid.fit(Xpol, Y)
Y_rid_est = rid.predict(Xpol)

# plot results
plt.plot(X, pol_fun(X), label='true fun')
plt.scatter(X, Y, label='noisy data', c='k', marker='.')
plt.plot(X, Y_est, label='lin.reg.')
plt.plot(X, Y_rid_est, label='ridge reg.')
plt.legend()

# estimated theta
print('theta_lr  = {}, {}'.format(lr.intercept_, lr.coef_[0]))
print('theta_rid = {}, {}'.format(rid.intercept_, rid.coef_[0]))

## ii. Lasso regression


In [ ]:
# Try yourself!

# 01.G) Liner regression with gradient descent

Consider the following simple regression problem, where:
$$g(x) = 2 + {3 \over 10}  x$$

First we define $g(x)$ in Python, we get some data $y_i = g(x_i) + \eta$ and we use them to fit a model $f(x; \boldsymbol \theta) = \theta_0 + \theta_1 x$.

In [ ]:
def g(x):
    y = 2 + 0.3 * x
    return y

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(1233)

# create n observations
n = 20  # number of data points

# regressor
x = np.linspace(-1, 1, n)  
# noise
sigma = 0.1  # std of the noise
eta = np.random.normal(loc=0, scale=sigma, size=n)
# response
y = g(x) + eta

# plot
plt.plot(x, g(x), label='true fun', color ='red');         # real function
plt.scatter(x, y, label='noisy data', color='black');      # data affected by noise
plt.legend();

We can estimate $\hat{\boldsymbol \theta}$ using the tools that we saw in the first lab.

In [ ]:
from sklearn.linear_model import LinearRegression

x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

model = LinearRegression()
model.fit(x, y)

theta_hat = np.c_[model.intercept_, model.coef_[0]]
print('theta_hat = {}'.format(theta_hat.ravel()))
# estimated response
y_est = model.predict(x)

# plot
plt.plot(x, g(x), label='true fun', color='red');              # the real function
plt.scatter(x, y, label='noisy data', color='black');          # data affected by noise
plt.plot(x, y_est, label='est fun (sklearn)', color='green');  # estimate linear function
plt.legend();

During the lectures we have seen that, given a model $f(x; {\boldsymbol \theta})$ it is possible minimize the training error iteratively using gradient descent:


$${\boldsymbol \theta}^{i+1} \gets {\boldsymbol \theta}^i - \varepsilon_L \frac{\partial V_n({\boldsymbol \theta})}{\partial {\boldsymbol \theta}} \bigg \rvert_{{\boldsymbol \theta} = {\boldsymbol \theta}^i}$$

Consider the mean squared error, 
$$V_n({\boldsymbol \theta}) = {1 \over n}\sum_{i=1}^n\left(y_i - f(x_i; \boldsymbol \theta)\right)^2$$
then, in our case:
$$\frac{\partial V_n({\boldsymbol \theta})}{\partial {\boldsymbol \theta}} =
\left[ 
\begin{array}{c}
\frac{\partial V_n({\boldsymbol \theta})}{\partial {\theta_0}} \\
\frac{\partial V_n({\boldsymbol \theta})}{\partial {\theta_1}}
\end{array}
\right] =
\left[ 
\begin{array}{c}
-{2 \over n}\sum_{i=1}^n\left(y_i - f(x_i; \boldsymbol \theta)\right) \\
-{2 \over n}\sum_{i=1}^n\left(y_i - f(x_i; \boldsymbol \theta)\right)x_i
\end{array}
\right] = -{2 \over n}X^T(Y - X\boldsymbol \theta)$$

Let's do it in numpy.

In [ ]:
X = np.hstack((np.ones((x.shape[0], 1)), x)) # add a column of ones
y = y.reshape(-1,1)

In [ ]:
def V(X, theta, Y):
  return np.mean((Y - np.dot(X, theta))**2)

theta = np.array([[-10.], [-5.]]) # initial value theta, it is random you can change it
eps = .3  # step size
steps = 100 # number of GD steps
history = [theta.ravel().copy()]
errs = [V(X,theta,y)]

for _ in range(steps):  # (Note: underscore `_` is used as name for useless variables) 
  grad = - 2 * np.dot(X.T, (y - np.dot(X, theta))) / X.shape[0]
  theta = theta - eps * grad
  # log theta and loss
  history.append(theta.ravel().copy())
  errs.append(V(X,theta,y))


history = np.array(history)
print('theta_hat:', theta.ravel())

As you can see, the iterative procedure converge to the same values of the closed form solution.

Let's visualize the descent.

In [ ]:
# code for plotting

from mpl_toolkits.mplot3d import Axes3D

r = np.abs(history).max()
x_range = np.linspace(-r, r, 100)
y_range = np.linspace(-r, r, 100)

theta_0, theta_1 = np.meshgrid(x_range, y_range)
zs = np.array([V(X, t.reshape(-1,1), y) 
               for t in np.c_[np.ravel(theta_0), np.ravel(theta_1)]])

zs = zs.reshape(theta_0.shape)

fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(122)
ax1.set_xlabel(r'$\theta_0$')
ax1.set_ylabel(r'$\theta_1$')
c = ax1.contour(theta_0, theta_1, zs, levels=25,  cmap='viridis')
ax1.plot(history[:,0], history[:,1], '-x',color='red')

ax2 = fig.add_subplot(121, projection='3d')
ax2.plot_trisurf(theta_0.ravel(), theta_1.ravel(), zs.ravel(), cmap='viridis')
ax2.set_xlabel(r'$\theta_0$')
ax2.set_ylabel(r'$\theta_1$')
ax2.set_zlabel(r'$V(\theta)$')
ax2.plot(history[:,0], history[:,1], errs, '-x', color='red', alpha=1.)
plt.tight_layout();

You can try to play with the step size. What happens if you increase/decrease it?



# 01.H) More on linear regression
## i. Confidence intervals for the parameters

Assume that $X^\top X$ is invertible, then

$$
\hat \theta = X^+Y \sim N\big(\theta, \sigma_\eta^2 (X^\top X)^{-1}\big)
$$

$$
E[\hat \theta] = E[X^+Y] = X^+E[Y] = X^+ X\theta = (X^\top X)^{-1}X^\top X \theta = \theta 
$$

$$
Var[\hat \theta] = Var[X^+Y] = X^+Var[Y] (X^+)^\top = \sigma_\eta^2 (X^\top X)^{-1} X^\top X (X^\top X)^{-1} = \sigma_\eta^2 (X^\top X)^{-1} 
$$

A rule of thumb is the following

* Extract the diagonal from $\sigma_\eta^2 (X^\top X)^{-1}$, which gives you an idea of the variance of each component of $\theta$.
* For each component $\theta_i$, check if the interval $(\theta_i - 2\sigma_i, \theta_i + 2\sigma_i)$ contains the zero; if that is the case, we are not very confident that the $\theta_i\ne 0$, thus that $x_i$ is relevant in the model.